In [1]:
import nrrd
import numpy as np
import ndpatch
from glob import glob
import skimage
import os
from scipy import ndimage
import edt
import patchify

In [2]:
def z_padding(input):
    if input.shape[:2] == (512,512):
        pass
    else:
        input=input[:512,:512,:]
    z_size = 40-input.shape[2]%40
    pad = np.zeros(input.shape[0:2]+ (z_size,))
    return np.append(input, pad, axis=-1)

def gen_train_patches(train_dir, label_dir, out_im, out_la):
    overlap = 4
    data_list = glob('{}/*.nrrd'.format(train_dir))
    label_list = glob('{}/*.nrrd'.format(label_dir))

    for i, path in enumerate(data_list):
        filename = os.path.basename(path).split('.')[0]
        image, header = nrrd.read(path)
        im_deno = ndimage.median_filter(image, size=3)
        image_norm = (im_deno - np.min(im_deno))/(np.max(im_deno)-np.min(im_deno))
        img_ad_eq = skimage.exposure.equalize_adapthist(image_norm, clip_limit=0.03)
        label, head_l = nrrd.read(label_list[i])
        im_padded = z_padding(img_ad_eq)
        la_padded = z_padding(label)
        #rdm_la = edt.edt(la_padded, black_border=True)
        j=0
        patches_im=patchify.patchify(im_padded, (512,512,40), step=40).squeeze()
        patches_la=patchify.patchify(la_padded, (512,512,40), step=40).squeeze()
        for j in np.arange(patches_im.shape[0]):
            filename_im = out_im + filename + '_' + str(j)
            filename_la = out_la + filename + '_' + str(j)
            if float(np.max(patches_la[j,:,:,:])) == float(0):
                pass
            else:
                nrrd.write(filename_im + '.nrrd', patches_im[j,:,:,:])
                nrrd.write(filename_la + '.seg.nrrd', patches_la[j,:,:,:])

In [3]:
train_dir = './training'
label_dir = './ground'
out_im = './train_patch_z40_dnep_im/'
out_la = './train_patch_z40_dnep_la/'

gen_train_patches(train_dir, label_dir, out_im, out_la)

In [61]:
import patchify
import numpy as np
import nrrd
import tensorflow as tf
data, h = nrrd.read('./training/D1.nrrd')


In [62]:
data_z = z_padding(data).astype('int16')
patches = patchify.patchify(data_z, (512,512,48), step=32)
print(patches.shape)
patches = patches.squeeze()
print(patches.shape)

(1, 1, 8, 512, 512, 32)
(8, 512, 512, 32)


In [58]:
nrrd.write('padding_cut.nrrd', data_z)

In [1]:
import numpy as np
import nrrd
data,h =nrrd.read('./train_patch_z32_pad_im/D1_7.nrrd')

In [4]:
image, h = nrrd.read('./test/R1.nrrd')
im_deno = ndimage.median_filter(image, size=2)
image_norm = (im_deno - np.min(im_deno))/(np.max(im_deno)-np.min(im_deno))
img_ad_eq = skimage.exposure.equalize_adapthist(image_norm, clip_limit=0.03)

In [ ]:
import time
start = time.time()
im_frangi = skimage.filters.frangi(img_ad_eq, sigmas=(1,5,10))
end = time.time()
print('frangi filter time: ',(end-start))

In [ ]:
import itk
import time
start = time.time()
image, h = nrrd.read('./test/R1.nrrd')
im_deno = ndimage.median_filter(image, size=2)
image_norm = (im_deno - np.min(im_deno))/(np.max(im_deno)-np.min(im_deno))
img_ad_eq = skimage.exposure.equalize_adapthist(image_norm, clip_limit=0.03)
input = itk.GetImageFromArray(img_ad_eq)
hessian_img = itk.hessian_recursive_gaussian_image_filter(input, sigma=1.0)
vesselness_filter = itk.Hessian3DToVesselnessMeasureImageFilter[itk.ctype("float")].New()
vesselness_filter.SetInput(hessian_img)
vesselness_filter.SetAlpha1(5)
vesselness_filter.SetAlpha2(0.1)
output = itk.GetArrayFromImage(vesselness_filter)
print(np.max(output))
print(np.min(output))

nrrd.write('vessel_test_5_01_s1.nrrd', output)
end = time.time()
print('time: ', end-start)

In [11]:
import time
start = time.time()
vesselness_filter = itk.Hessian3DToVesselnessMeasureImageFilter[itk.ctype("float")].New()
vesselness_filter.SetInput(hessian_img)
vesselness_filter.SetAlpha1(0.5)
vesselness_filter.SetAlpha2(2.0)
output = itk.GetArrayFromImage(vesselness_filter)
print(np.max(output))
print(np.min(output))
nrrd.write('vessel_test_05_2.nrrd', output)
end = time.time()
print('time: ', end-start)


time:  0.0


In [12]:
start = time.time()
vesselness_filter.SetInput(hessian_img)
vesselness_filter.SetAlpha1(0.5)
vesselness_filter.SetAlpha2(2.0)
end = time.time()

In [13]:
output = itk.GetArrayFromImage(vesselness_filter)
print(np.max(output))
print(np.min(output))

909.08215
0.0


In [14]:
nrrd.write('vessel_test.nrrd', output)